In [1]:
from sklearn.datasets import fetch_20newsgroups
#Choosing categories we require 
categories = ['alt.atheism','soc.religion.christian','comp.graphics','sci.med']
#Will take data from the folder train/test
news_train = fetch_20newsgroups(subset='train',categories=categories,shuffle=True)
news_test = fetch_20newsgroups(subset='test',categories=categories,shuffle=True)

In [3]:
import nltk
nltk.download()
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import NearestCentroid
import numpy as np

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [4]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                             ('tfidf', TfidfTransformer()),
                             ('mnb', NearestCentroid()),])

text_mnb_stemmed = text_mnb_stemmed.fit(news_train.data, news_train.target)

predicted_mnb_stemmed = text_mnb_stemmed.predict(news_test.data)

np.mean(predicted_mnb_stemmed == news_test.target)

0.8501997336884154

In [5]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(news_test.target,predicted_mnb_stemmed))
print(metrics.classification_report(news_test.target,predicted_mnb_stemmed,target_names=news_test.target_names)),metrics.confusion_matrix(news_test.target,predicted_mnb_stemmed)

Accuracy: 0.8501997336884154
                        precision    recall  f1-score   support

           alt.atheism       0.92      0.66      0.77       319
         comp.graphics       0.78      0.99      0.87       389
               sci.med       0.91      0.83      0.87       396
soc.religion.christian       0.84      0.89      0.87       398

              accuracy                           0.85      1502
             macro avg       0.86      0.84      0.84      1502
          weighted avg       0.86      0.85      0.85      1502



(None,
 array([[210,  26,  23,  60],
        [  1, 384,   4,   0],
        [  6,  56, 327,   7],
        [ 11,  27,   4, 356]], dtype=int64))